In [1]:
import time

import numpy as np
from joblib import Parallel, delayed
from numba import jit
from sklearn.metrics.pairwise import pairwise_distances, pairwise_kernels


def compute_kernel(X, sigma):
    """
    X : (n_samples, n_dim)
    sigma : float
    """

    d = X.shape[1]
    denom = np.power(2 * np.pi, d / 2.0) * np.power(sigma, d / 2)
    constant = 1 / denom

    kern = pairwise_kernels(X, metric="rbf", gamma=1 / (sigma * 2)) * constant

    return kern

In [50]:
cov = np.array([[1, 0.7, 0.6], [0.7, 1, 0.6], [0.6, 0.6, 1]])

In [51]:
cov

array([[1. , 0.7, 0.6],
       [0.7, 1. , 0.6],
       [0.6, 0.6, 1. ]])

In [42]:
def example1(n, reps):
    mean = np.array([0, 0, 0])
    cov = np.array([[1, 0.36, 0.6], [0.36, 1, 0.6], [0.6, 0.6, 1]])

    X = np.random.multivariate_normal(mean, cov, size=(reps, n))
    Y = np.random.multivariate_normal(mean, cov, size=(reps, n))
    Z = np.random.multivariate_normal(mean, cov, size=(reps, n))

    return X, Y, Z


def example5(n, reps):
    mean = np.array([0, 0, 0])
    cov = np.array([[1, 0.7, 0.6], [0.7, 1, 0.6], [0.6, 0.6, 1]])

    X = np.random.multivariate_normal(mean, cov, size=(reps, n))
    Y = np.random.multivariate_normal(mean, cov, size=(reps, n))
    Z = np.random.multivariate_normal(mean, cov, size=(reps, n))

    return X, Y, Z

In [2]:
def weighted_center_mat(distx, weights):
    n = distx.shape[0]

    weight_distance = np.average(distx, axis=0, weights=weights)
    weight_distance_sum = np.average(weight_distance, weights=weights)

    exp_distx = (
        np.repeat(weight_distance, n).reshape(-1, n).T
        + np.repeat(weight_distance, n).reshape(-1, n)
        - (weight_distance_sum)
    )

    cent_distx = distx - exp_distx

    return cent_distx


def weighted_center_mat2(distx, weights):
    n = distx.shape[0]

    row_sum = np.average(distx, axis=0, weights=weights)
    total_sum = np.average(row_sum, weights=weights)

    # exp_distx = (
    #     np.repeat(weight_distance, n).reshape(-1, n).T
    #     + np.repeat(weight_distance, n).reshape(-1, n)
    #     - (weight_distance_sum)
    # )

    cent_distx = distx - row_sum.reshape(-1, n).T - row_sum.reshape(-1, n) + total_sum

    return cent_distx

In [3]:
@jit(nopython=True)
def weighted_center_matjit(distx, weights):
    n = distx.shape[0]

    scl = np.sum(weights)
    row_sum = np.sum(np.multiply(distx, weights), axis=1) / scl
    total_sum = weights @ row_sum / scl

    exp_distx = (
        np.repeat(row_sum, n).reshape(-1, n).T
        + np.repeat(row_sum, n).reshape(-1, n)
        - (total_sum)
    )

    cent_distx = distx - exp_distx

    return cent_distx


@jit(nopython=True)
def weighted_center_mat2jit(distx, weights):
    n = distx.shape[0]

    scl = np.sum(weights)
    row_sum = np.sum(np.multiply(distx, weights), axis=1) / scl
    total_sum = weights @ row_sum / scl

    # exp_distx = (
    #     np.repeat(weight_distance, n).reshape(-1, n).T
    #     + np.repeat(weight_distance, n).reshape(-1, n)
    #     - (weight_distance_sum)
    # )

    cent_distx = distx - row_sum.reshape(-1, n).T - row_sum.reshape(-1, n) + total_sum

    return cent_distx

In [36]:
def compute_dcov(distx, disty, distz):
    n = distx.shape[0]

    cdcov = np.zeros(n)

    for i in range(n):
        r = distz[[i]]
        cdx = weighted_center_mat2(distx, distz[i])
        cdy = weighted_center_mat2(disty, distz[i])
        cdcov[i] = (cdx * cdy * r * r.T).sum() / r.sum() ** 2

    cdcov *= 12 * np.power(distz.mean(axis=0), 4)

    return cdcov.mean()


def resample_index(probs, axis=1):
    n = probs.shape[1 - axis]
    sums = probs.sum(axis=1, keepdims=True)
    idx = ((probs / sums).cumsum(axis=1) > np.random.rand(n)[:, None]).argmax(axis=1)

    return idx


def bootstrap(distx, disty, distz):
    idx = resample_index(distz)
    permx = distx[idx][:, idx]

    permuted_stat = compute_dcov(permx, disty, distz)

    return permuted_stat


def cdcov_test(X, Y, Z, sigma=1, reps=1000, workers=1):
    distx = pairwise_distances(X)
    disty = pairwise_distances(Y)
    distz = compute_kernel(Z, sigma=sigma)

    cdcov_stat = compute_dcov(distx, disty, distz)

    permuted_stats = Parallel(n_jobs=workers)(
        delayed(bootstrap)(distx, disty, distz) for _ in range(reps)
    )
    permuted_stats = np.array(permuted_stats)

    pvalue = (1 + (permuted_stats >= cdcov_stat).sum()) / (1 + reps)

    return pvalue

In [18]:
X = np.loadtxt("./data/x.csv", skiprows=1, delimiter=",")
Y = np.loadtxt("./data/y.csv", skiprows=1, delimiter=",")
Z = np.loadtxt("./data/z.csv", skiprows=1, delimiter=",")

distx = pairwise_distances(X)
disty = pairwise_distances(Y)
distz = compute_kernel(Z, 0.5)

In [33]:
n = 100
reps = 100

X, Y, Z = example1(n, reps)

np.save(f"./data/x_{n}", X)
np.save(f"./data/y_{n}", Y)
np.save(f"./data/z_{n}", Z)

In [37]:
reps = 100
bootstraps = 250

ns = [50, 100, 150, 200, 250]
sigma = 1

results = {}


tmp = []
for i in range(reps):
    tmp.append(cdcov_test(X[i], Y[i], Z[i], sigma=1 / 3, reps=500, workers=-1))

In [40]:
(np.array(tmp) < 0.05).mean()

0.04

In [43]:
n = 100
reps = 100

X, Y, Z = example5(n, reps)

In [44]:
tmp = []
for i in range(reps):
    tmp.append(cdcov_test(X[i], Y[i], Z[i], sigma=1 / 3, reps=500, workers=-1))

(np.array(tmp) < 0.05).mean()

0.03

In [48]:
i = 5

In [49]:
cdcov_test(X[i], Y[i], Z[i], sigma=1 / 3, reps=1000, workers=1)

0.6973026973026973

In [52]:
mean = np.array([0, 0, 0])
cov = np.array([[1, 0.7, 0.6], [0.7, 1, 0.6], [0.6, 0.6, 1]])

X = np.random.multivariate_normal(mean, cov, size=n)
Y = np.random.multivariate_normal(mean, cov, size=n)
Z = np.random.multivariate_normal(mean, cov, size=n)

In [57]:
cdcov_test(X, Y, Z, sigma=1 / n, reps=1000, workers=1)

0.10789210789210789

In [56]:
8 * np.sqrt(np.pi) / 3

4.726543602414709